In [1]:
# 导入专利可视化程序包
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# 读取数据为DataFrame
gaa = pd.read_excel('gaa.xlsx', 
                    header=0, 
                    index_col=0, 
                    parse_dates=[2, 6, 28, 29, 162, 168, 169, 173, 174])

# 第一部分 专利态势分析
所谓专利态势分析，即对某一行业或技术领域的专利总体状况进行分析，通常是针对专利数量进行统计分析。基本的专利数量统计分析包括：
* 申请趋势分析
* 技术构成分析
* 地域分布分析
* 申请人排名分析

## 一、申请趋势分析
申请趋势分析常见的分析对象：
* 申请量
* 授权量
* 有效量
* 发明人数量
* 申请人数量
  
申请趋势分析常用图表：
* 折线图
* 面积图
* 柱形图
  
其中，折线图和面积图侧重表示较长时间段内的数量变化趋势，而柱形图侧重表示在短时间段内的数量变化情况，并突出每一个时间段的具体数量值。

### 1. 折线图
#### （1）标准折线图